### 국내 주식 주별 시세 UPDATE

In [4]:
import requests
import json
import pandas as pd
from datetime import datetime
from dotenv import load_dotenv
import os

In [31]:
url = "/uapi/domestic-stock/v1/quotations/inquire-daily-itemchartprice"

load_dotenv()

my_app= os.getenv("MY_APP")
my_sec= os.getenv("MY_SEC")

#계좌번호 앞 8자리
my_acct="43016876"
my_acct_stock= "증권계좌 8자리"
my_acct_future= "선물옵션계좌 8자리"
#계좌번호 뒤 2자리
my_prod= "01"
#my_prod: "03"

#실전투자
prod= "https://openapi.koreainvestment.com:9443"
#모의투자
vps= "https://openapivts.koreainvestment.com:29443"


my_agent= "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"

class Env:
    def __init__(self):
        self.my_app = my_app
        self.my_sec = my_sec
        self.my_url = prod
        self.my_token = None
        self.my_acct = my_acct
        self.my_prod = my_prod
_env = Env()

In [57]:
def auth():
    url = f'{_env.my_url}/oauth2/tokenP'
    headers = {"content": "application/json"}
    data = {
        "grant_type":"client_credentials",
        "appkey": _env.my_app,
        "appsecret": _env.my_sec
    }

    res = requests.post(url, headers=headers, data= json.dumps(data))
    if res.status_code == 200:
        print('토큰 발급 성공')
        _env.my_token = res.json()['access_token']
    else:
        print("토큰 발급 실패")
        print(res.status_code, res.text)

In [58]:
auth()

토큰 발급 성공


KOSPI200 종목에 대해 주별 시세 받아오기

In [9]:
stock_code = "005930"
market_code = "J"

In [59]:
headers = {
    "Content-Type": "application/json",
    "authorization": f'Bearer {_env.my_token}',
    "appkey": _env.my_app,
    "appsecret": _env.my_sec,
    "tr_id": "FHKST03010100", 
    "custtype": "P"
}

In [99]:
params = {
    "fid_cond_mrkt_div_code": market_code,  
    "fid_input_iscd": stock_code,
    "fid_period_div_code": "W"
}

In [100]:
START = 2015
END = 2024

In [101]:
params['fid_input_date_1'] = '20240101'
params['fid_input_date_2'] = '20241231'
params['fid_org_adj_prc'] = 0

In [102]:
print(params)

{'fid_cond_mrkt_div_code': 'J', 'fid_input_iscd': '005930', 'fid_period_div_code': 'W', 'fid_input_date_1': '20240101', 'fid_input_date_2': '20241231', 'fid_org_adj_prc': 0}


In [103]:
my_url=_env.my_url+url
response = requests.get(my_url, headers=headers, params=params)
if response.status_code != 200:
    print('조회 실패')
else:
    print('조회 성공')
    data = response.json()

조회 성공


In [104]:
response.headers

{'Date': 'Thu, 17 Apr 2025 19:33:52 GMT', 'X-Content-Type-Options': 'nosniff', 'Content-Length': '15413', 'X-ORACLE-DMS-ECID': '006CjtvMBfPFw0zzzR06zzEjX2E0jYYz30002f3008zGW', 'tr_cont': '', 'tr_id': 'FHKST03010100', 'gt_uid': 'e50c78341bc211f090aa4b32bcc37055', 'X-ORACLE-DMS-RID': '0:1', 'X-XSS-Protection': '1; mode=block', 'Keep-Alive': 'timeout=5, max=100', 'Connection': 'Keep-Alive', 'Content-Type': 'application/json'}

In [123]:
li = response.json()['output2']
df = pd.DataFrame(li)
df.sort_values(by='stck_bsop_date')
df = df[['stck_bsop_date', 'stck_clpr', 'stck_oprc', 'stck_hgpr', 'stck_lwpr', 'acml_vol', 'acml_tr_pbmn']]

df = df.rename(columns={
    "stck_bsop_date": "date",
    "stck_clpr": "close",
    "stck_oprc": "open",
    "stck_hgpr": "high",
    "stck_lwpr": "low",
    "acml_vol": "volume",
    "acml_tr_pbmn": "trade_amount"
})
df

,date,close,open,high,low,volume,trade_amount
0,20241230,54400,53300,55100,52300,48573286,2601445342122
1,20241223,53700,53400,54600,53200,46571598,2506960694800
2,20241216,53000,56300,56600,51900,96348143,5198145218285
3,20241209,56100,53500,56500,53200,110963118,6060547226707
4,20241202,54100,54300,54400,52000,120874387,6478011775994
5,20241125,54200,57400,58900,53800,125769782,7118594343240
6,20241118,56000,57000,57500,54700,134877926,7597773828968
7,20241111,53500,56700,56800,49900,215587402,11326757742256
8,20241104,57000,58600,59400,56300,86084137,4968496206352
9,20241028,58300,55700,61200,55700,130875210,7702242715876


In [115]:
raw = response.json()['output1']

parsed = {
    "code": raw["stck_shrn_iscd"],
    "face_value": int(raw["stck_fcam"]),
    "listed_shares": int(raw["lstn_stcn"]),
    "capital": int(raw["cpfn"]),
    "market_cap": int(raw["hts_avls"]),
    "per": float(raw["per"]),
    "eps": float(raw["eps"]),
    "pbr": float(raw["pbr"]),
    "loan_balance_ratio": float(raw["itewhol_loan_rmnd_ratem name"])
}

parsed

{'code': '005930',
 'face_value': 100,
 'listed_shares': 5919637922,
 'capital': 7780,
 'market_cap': 3261720,
 'per': 11.13,
 'eps': 4950.0,
 'pbr': 0.95,
 'loan_balance_ratio': 0.22}